In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
import optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, roc_curve, auc, log_loss

In [2]:
X = pd.read_pickle(r'C:\Users\domin\Documents\ML projects\AE-prediction\train_v2.pkl')

In [3]:
enc = LabelEncoder()
for col in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']:
    try:
        X[col] = enc.fit_transform(X[col])
    except:
        pass
    try:
        test[col] = enc.transform(test[col])
    except:
        pass
X = X.fillna(-1)
y = X['target']
X = X.drop(['target'],axis=1)

In [4]:
print('Shape of X: ' + str(X.shape))
print('Shape of y: ' + str(y.shape))

Shape of X: (458913, 161)
Shape of y: (458913,)


In [5]:
def objective(trial, X, y):
    param_grid = {
        #'criterion' : trial.suggest_categorical('criterion', ['gini', 'entropy']),
        #'bootstrap' : trial.suggest_categorical('bootstrap',['True','False']),
        'n_estimators' : trial.suggest_int("n_estimators", 100, 1400, step=100),
        'max_depth' : trial.suggest_int("max_depth", 3, 12),
        'min_samples_split' : trial.suggest_int("min_samples_split", 200, 1000, step=100),
        'min_samples_leaf' : trial.suggest_int("min_samples_leaf", 200, 1000, step=100),
        'max_features' : trial.suggest_categorical("max_features", ['sqrt', 'log2', None])    
    }
    
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    cv_scores = np.empty(10)
    
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = RandomForestClassifier(random_state=0, **param_grid)
        model.fit(
            X_train,
            y_train)
        
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)
    return np.mean(cv_scores)

In [ ]:
study = optuna.create_study(direction="minimize", study_name="RF classifier")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=100)

[I 2022-08-13 09:16:49,854] A new study created in memory with name: RF classifier


In [ ]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")
for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
param_grid = {
        #'criterion' : trial.suggest_categorical('criterion', ['gini', 'entropy']),
        #'bootstrap' : trial.suggest_categorical('bootstrap',['True','False']),
        'n_estimators' : 1200,
        'max_depth' : 4,
       # 'min_samples_split' : 300,
       # 'min_samples_leaf' : 150,
       # 'max_features' : None
    }

model = RandomForestClassifier(random_state=0, **param_grid)
model.fit(
            X_train,
            y_train)
preds = model.predict_proba(X_test)
print(log_loss(y_test, preds))

0.2820662001765465
